<a href="https://colab.research.google.com/github/JiUUoong/CODE/blob/main/actorcritic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# actor critic
# 하이브리드 알고리즘
# 정책 기반으로 개선되는 actor
# 가치 함수 기반으로 개선되는 critic
# critic의 비평 (Q(s, a)): 방금 actor에 의해 선택된 행동은 얼만큼의 가치가 있다
# actor의 행동 선정은 정책을 따름: 이 정책의 개선은 policy gradient theorem을 따름

In [ ]:
import gym # CartPole 환경 이번엔 AC 알고리즘으로
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers   # import torch.nn as nn; 예를 들면 layers.Dense가 nn.Linear

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [ ]:
seed = 42
env = gym.make("CartPole-v1")
state = env.reset(seed=seed)
np.random.seed(seed)
tf.random.set_seed(seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
# 하이퍼파라미터
GAMMA = 0.99
EPS = np.finfo(np.float32).eps.item()   # numpy 상에서 정해져있는 가장 작은 수: 1.0 + eps != 1.0
LR = 0.01

In [ ]:
# 한 에피소드에서 최대 버틸 수 있는 스텝 수
max_steps = 1000
state = env.reset()
state.shape[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


4

In [ ]:
n_observations = state.shape[0]
n_actions = env.action_space.n
# 신경망 모델
# actor, critic 각각 신경망 만들기
# 입력 층
inputs = layers.Input(shape=(n_observations,))
# 공통 층
n_hidden = 128
common = layers.Dense(n_hidden, activation="relu")(inputs)
common2 = layers.Dense(n_hidden, activation="relu")(common)
# actor critic 각각의 마지막 층
# actor
actor = layers.Dense(n_actions, activation="softmax")(common)
# critic
critic = layers.Dense(1)(common)
model = keras.Model(inputs=inputs, outputs=[actor, critic])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 4)]                  0         []                            
                                                                                                  
 dense_20 (Dense)            (None, 128)                  640       ['input_6[0][0]']             
                                                                                                  
 dense_22 (Dense)            (None, 2)                    258       ['dense_20[0][0]']            
                                                                                                  
 dense_23 (Dense)            (None, 1)                    129       ['dense_20[0][0]']            
                                                                                            

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=LR)     # torch.optim.Adam()
huber_loss = keras.losses.Huber()                       # torch.nn.HuberLoss()

In [ ]:
# keras에서 모델 만들고 학습시키기 준비하는 과정(방법)
# 1. 모델 정의, optimizer 정의, loss 정의 -> 이후 model.compile(optimizer, loss)
# 2. GradientTape 스코프 안에서 학습 with gradienTape as gt:

In [ ]:
# 기록을 위한 몇개 빈 리스트
action_probs = []     # 행동을 고를 확률
critic_value = []     # 행동 가치
rewards = []          # 보상
reward_ma = 0         # reward Moving Average
ep = 0                # 지금 몇번째 에피소드 학습 중인지

while True:           # 풀릴 때까지 (180스텝)
  state = env.reset()
  ep_reward = 0       # 이번 에피소드에 얻은 보상
  with tf.GradientTape() as tape:
    for timestep in range(1, max_steps):
      state = tf.convert_to_tensor(state) # torch.tensor(배열이름)
      state = tf.expand_dims(state, 0)    # 텐서이름.unsqueeze(0)
      # 모델에게 state를 주고 출력값 받기
      # 103번 슬라이드
      action_dist, q_value = model(state)
      # 출력된 정책 분포에 따라 액션 고르기
      # action_dist는 0번째 축이 추가되어있음 (1, 2) -> (2,)
      # n_actios개의 선택지 중에 샘플링하겠다 샘플링 확률은 p에 따라
      action = np.random.choice(n_actions, p=np.squeeze(action_dist))
      # 96번 슬라이드 액터 부분 (log (pi))
      action_probs.append(tf.math.log(action_dist[0, action]))
      # (1, 1)
      critic_value.append(q_value[0, 0])
      # 104번 슬라이드
      state, reward, terminated, truncated = env.step(action)
      rewards.append(reward)
      ep_reward += reward
      # 태스크 종료되면 이번 에피소드 (안쪽 루프)에서 나오기
      if terminated or truncated:
        break
    reward_ma = 0.01 * ep_reward + (1-0.01) * reward_ma
    # 에피소드 단위의 내용 return
    returns = []
    discounted_sum = 0
    for r in rewards[::-1]:   # rewards 리스트 거꾸로
      discounted_sum = r + GAMMA * discounted_sum
      returns.insert(0, discounted_sum)
    # return(G_t) 값 정규화 (normalize) sklearn.preprocessing StandardScaler()
    returns = np.array(returns)   # list를 numpy 배열로
    returns = (returns - np.mean(returns)) / np.std(returns) + EPS
    returns = returns.tolist()    # 다시 list로

    # 손실 값 계산
    history = zip(action_probs, critic_value, returns)
    actor_losses = []
    critic_losses = []
    for log_prob, value, ret in history:
      # 오차
      diff = ret - value    # G_t - Q(s, a)
      actor_losses.append(-log_prob * diff)
      critic_losses.append(
          huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
      )
    loss_value = sum(actor_losses) + sum(critic_losses)
    grads = tape.gradient(loss_value, model.trainable_variables)          # loss.backward()
    # 기울기 계산: 손실 값에 기여하는 변수가 여럿임 : 모델의 파라미터들 (model.trainable_variables)
    # 이 각 변수에 대해 미분 (편미분)해서 각 변수의 기여도를 수치화하고
    # 그 수치에 비례하게 보정
    optimizer.apply_gradients(zip(grads, model.trainable_variables))      # optimizer.step()

    # 이번 에피소드 관련 모든 기록 초기화
    action_probs.clear()
    critic_value.clear()
    rewards.clear()
  # GradienTape 스코프에서 나오기
  ep += 1
  if ep % 10 == 0:
    print("보상 이동평균: {:.2f}. 에피소드 {} 학습 중".format(reward_ma, ep))
  if reward_ma > 180:
    print("학습 완료")
    break

보상 이동평균: 3.70. 에피소드 10 학습 중
보상 이동평균: 7.61. 에피소드 20 학습 중
보상 이동평균: 11.06. 에피소드 30 학습 중
보상 이동평균: 15.93. 에피소드 40 학습 중
보상 이동평균: 18.00. 에피소드 50 학습 중
보상 이동평균: 20.83. 에피소드 60 학습 중
보상 이동평균: 24.42. 에피소드 70 학습 중
보상 이동평균: 25.43. 에피소드 80 학습 중
보상 이동평균: 29.31. 에피소드 90 학습 중
보상 이동평균: 30.33. 에피소드 100 학습 중
보상 이동평균: 38.13. 에피소드 110 학습 중
보상 이동평균: 50.34. 에피소드 120 학습 중
보상 이동평균: 64.26. 에피소드 130 학습 중
보상 이동평균: 87.15. 에피소드 140 학습 중
보상 이동평균: 100.66. 에피소드 150 학습 중
보상 이동평균: 116.74. 에피소드 160 학습 중
보상 이동평균: 121.34. 에피소드 170 학습 중
보상 이동평균: 144.75. 에피소드 180 학습 중
보상 이동평균: 154.11. 에피소드 190 학습 중
보상 이동평균: 145.46. 에피소드 200 학습 중
보상 이동평균: 148.91. 에피소드 210 학습 중
보상 이동평균: 178.44. 에피소드 220 학습 중
학습 완료
